# Exploratory Data Analysis
----



### Lighthouse Labs, Midterm Project Project - Predicting Flight Delays.

##### January 13, 2023. Terre Leung, Tetiana Fesenko, and Jamie Dormaar

---

_Use this notebook to get familiar with the datasets we have. There is 10 questions we need to answer during the EDA._


_We shouldn't limit our EDA to these 10 questions. Let's be creative :)._

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme()

from scipy import stats

import warnings
warnings.filterwarnings('ignore')

import os
import json
import requests



In [135]:
# Load data tables:
# flights_initial_500000_records    = pd.read_csv('../data/flights_initial_500000_records.csv', delimiter= ',')
# flights_delay_dates_all_records   = pd.read_csv('../data/flights_delay_dates_all_records.csv', delimiter= ',')
# flights_random_100000_records     = pd.read_csv('../data/flights_random_100000_records.csv', delimiter= ',')
# flights_random_5000_records       = pd.read_csv('../data/flights_random_5000_records.csv', delimiter= ',')
# flights_test_all_records          = pd.read_csv('../data/flights_test_all_records.csv', delimiter= ',')
# fuel_consumption_all_records      = pd.read_csv('../data/fuel_consumption_all_records.csv', delimiter= ',')
passengers_initial_300000_records = pd.read_csv('../data/passengers_initial_300000_records.csv', delimiter= ',')
# flights_usa = pd.read_csv('../data/usa_flights2.csv', delimiter= ',')  # Terre is there a new csv to go with this one?
flights_usa = pd.read_csv('../data/flights_random_100000_records.csv', delimiter= ',')

In [ ]:
# Save working copies of the data:
df_fl_init    = flights_initial_500000_records.copy()
df_fl_delays  = flights_delay_dates_all_records.copy()
df_fl_smpl1   = flights_random_100000_records.copy()
df_fl_smpl2   = flights_random_5000_records.copy()
df_fl_test    = flights_test_all_records.copy()
df_fc         = fuel_consumption_all_records.copy()
df_pa_init    = passengers_initial_300000_records.copy()

##### Set your session working table to temp variable df:

In [ ]:
df = df_fl_smpl1.copy()

##### SAVE a session timestamp to label the saved outputs: 
>(Optional: this can be useful if you want to help keep your files organized)


In [ ]:

tag = 'smpl_100K_' # option with leading name            'Jamie_'
# tag = ''
# dt = dt.now().time().strftime(f'%b%d_%H%M')   # 'Jan01_1704'
# dt = dt.now().time().strftime(f'%b%-d_%H%M')  # 'Jan1_1708'
# dt = dt.now().time().strftime(f'%a_%H%M')        # 'Mon_1710'
dt = ''
session = f'{tag}{dt}'

##### SETUP: A first look at tables:


In [ ]:

# flights_initial_500000_records
print(f'\nflights_initial_500000_records.shape: {flights_initial_500000_records.shape}')
display(flights_initial_500000_records.head())

# flights_delay_dates_all_records
print(f'\nflights_delay_dates_all_records.shape: {flights_delay_dates_all_records.shape}')
display(flights_delay_dates_all_records.head())

# flights_random_100000_records
print(f'\nflights_random_100000_records.shape: {flights_random_100000_records.shape}')
display(flights_random_100000_records.head())

# flights_test_all_records
print(f'\nflights_test_all_records.shape: {flights_test_all_records.shape}')
display(flights_test_all_records.head())

# fuel_consumption_all_records
print(f'\nfuel_consumption_all_records.shape: {fuel_consumption_all_records.shape}')
display(fuel_consumption_all_records.head())

# passengers_initial_300000_records
print(f'\npassengers_initial_300000_records.shape: {passengers_initial_300000_records.shape}')
display(passengers_initial_300000_records.head())


In [ ]:
df.info()

##### NOTE: Missing Data content for each of the four data tables.


In [ ]:
# Check for nulls:
# flights Table percent Null content:
df_fl_init_nulls = df_fl_init.isnull().sum().sort_values(ascending= False)
percent = (df_fl_init.isnull().sum()/df_fl_init.isnull().count()).sort_values(ascending = False)
df_fl_init_missing_data = pd.concat(
    [df_fl_init_nulls, percent]
  , axis=1
  , keys=['Total', 'Percent']
  , verify_integrity= True
)
print(f'\nflights_missing_data.head(20)')
display(df_fl_init_missing_data.head(20))

# flights_test Table percent Null content:
df_fl_test_nulls = df_fl_test.isnull().sum().sort_values(ascending= False)
percent = (df_fl_test.isnull().sum()/df_fl_test.isnull().count()).sort_values(ascending = False)
df_fl_test_missing_data = pd.concat(
    [df_fl_test_nulls, percent]
  , axis=1
  , keys=['Total', 'Percent']
  , verify_integrity= True
)
print(f'\nflights_test_missing_data.head(20)')
display(df_fl_test_missing_data.head(20))

# fuel_consumption Table percent Null content:
df_fc_nulls = df_fc.isnull().sum().sort_values(ascending= False)
percent = (df_fc.isnull().sum()/df_fc.isnull().count()).sort_values(ascending = False)
df_fc_missing_data = pd.concat(
    [df_fc_nulls, percent]
  , axis=1
  , keys=['Total', 'Percent']
  , verify_integrity= True
)
print(f'\nfuel_consumption_missing_data.head(20)')
display(df_fc_missing_data.head(20))

# passengers Table percent Null content:
df_pa_init_nulls = df_pa_init.isnull().sum().sort_values(ascending= False)
percent = (df_pa_init.isnull().sum()/df_pa_init.isnull().count()).sort_values(ascending = False)
df_pa_init_missing_data = pd.concat(
    [df_pa_init_nulls, percent]
  , axis=1
  , keys=['Total', 'Percent']
  , verify_integrity= True
)
print(f'\npassengers_missing_data.head(20)')
display(df_pa_init_missing_data.head(20))


##### NOTE: Differences between flights, and flights_test table data:

In [ ]:
flights_columns = df_fl_init.columns
flights_columns

In [ ]:
flights_test_columns = df_fl_test.columns
flights_test_columns

In [ ]:
fl_test_exclusion = df_fl_init[df_fl_init.columns[~df_fl_init.columns.isin([flights_test_columns])]]
fl_test_exclusion.head()

##### ANALYZE: Arrival delay details in the flights table:


#### **Task 1**: 

1. Test the hypothesis that the delay is from Normal distribution. 
1. And, that the **mean** of the arrival delays is 0. 
1. Be careful about the outliers.

>##### TASK 1.1: Test the hypothesis that the delay is from Normal distribution. 

In [ ]:
df[['arr_delay']].value_counts().sort_values(ascending=False).head(20)

In [ ]:
df['arr_delay'].describe()

The `stats` package from the `scipy` module will test the Null hypothesis that the data is normally distributed.
If the resulting p value is > than 0.05 we can assume the data is distributed normally with high statistical probability.

In [ ]:
# from scipy import stats
stat, p = stats.shapiro(df['arr_delay'])
print('%0.15f' % p, stat)

The statistical calculation above printed the following warning:
```
UserWarning: p-value may not be accurate for N > 5000.
```
Smaller samples taken to correct for this inaccuracy:

In [ ]:
# Sample a subset:
x = df.sample(1000)
len(x)

In [ ]:
# Rerun Shapiro Wilk Normality Test:
stat, p = stats.shapiro(x['arr_delay'])
print('%0.15f' % p, stat)

The data appears to indeed be normally distributed.

>##### TASK 1.3: Managing outliers.

##### VISUALIZE: Arrival delay distribution, and manage outliers:

In [ ]:
plt.hist(df['arr_delay'], bins=500)


plt.xlabel('Delay Time (min)')
plt.title('Arrival Delay Distribution')
plt.xlim(-50, 50)
plt.ylim(0, 10000)

plt.savefig(f'../Images/Arrival_delay_distn_{session}.png')
plt.show()


In [ ]:
# Outlier detection:
sns.boxplot(data= df, x='arr_delay', whis= 2.5)

plt.savefig(f'../Images/Arrival_delay_outliers_boxplot_{session}.png')

Manually chosen outlier range limits:

In [ ]:
# # Define and remove the outliers by a chosen parameter:
# max_delay = 100
# outliers    = df_fl_init[df_fl_init['arr_delay'] > max_delay]
# df_fl_clean = df_fl_init[df_fl_init['arr_delay'] < max_delay]

or using the standard 1.5 * IQR:

In [ ]:
# Instantiate the Arrival Delays:
delays = df['arr_delay']

# Define the quantiles of the delay distribution:
Q1 = delays.quantile(0.25)
Q3 = delays.quantile(0.75)
IQR = Q3 - Q1

# Define the outlier thresholds
min_threshold = (Q1 - 1.5 * IQR)
max_threshold = (Q3 + 1.5 * IQR)

In [ ]:
df_clean = df[~((delays < min_threshold)|(delays > max_threshold))]
df_clean.shape

In [ ]:
sns.boxplot(x=df_clean['arr_delay'])

#SAVE boxplot of clean delay distribution:
plt.savefig(f'../Images/Arrival_delay_boxplot_{session}.png')

In [ ]:
# fig, ax1 = plt.subplots()

# ax1 = fl_df_clean.plot()
# ax2 = fl_df.plot()

# ax1.hist([y1, y2])
# ax1.set_xlim(-10,10)
# fig, (ax1, ax2) = plt.subplots(1, 2)


#### **Task 2**: Is average/median monthly delay different during the year? If yes, which are months with the biggest delays and what could be the reason?

In [ ]:
# Convert fl_date from string to datetime data type
df_clean[['fl_date']] = df_clean[['fl_date']].apply(pd.to_datetime)

In [ ]:
df_clean['date'] = df_clean['fl_date'].dt.date
df_clean['year'] = df_clean['fl_date'].dt.year
df_clean['month'] = df_clean['fl_date'].dt.month


In [ ]:
print(df_clean[['year', 'month', 'fl_date', 'date']].dtypes)
display(df_clean[['year', 'month', 'fl_date', 'date']].head())

In [ ]:
df_clean[['month', 'year', 'arr_delay']].groupby(['year', 'month']).describe()

>NOTE: This was the point where we discovered that our initial sample of 500000 records we collected from the source flights table turned out to only include records from 2 months in 2018.  So evidently the source table is sorted by date.

In [ ]:
df_clean.columns

In [ ]:
# Separate the data for easier viewing re annual delay trends:
df_clean_2018 = df_clean[df_clean['year']==2018]
df_clean_2019 = df_clean[df_clean['year']==2019]

In [ ]:
df_clean_2018.groupby(['month']).agg({'arr_delay': np.mean}).sort_values('arr_delay', ascending=False)
# df_delays_2018.groupby(['year', 'month']).agg(({'arr_delay': np.median}))

In [ ]:
df_clean_2019.groupby('month').agg({'arr_delay': np.mean}).sort_values('arr_delay', ascending=False)
# df_delays_2019.groupby(['year', 'month']).agg(({'arr_delay': np.median}))

In [ ]:
sns.scatterplot(data=df_clean_2019, x="month", y="arr_delay")
plt.show()

In [ ]:
# month
var = 'month'
data = df_clean_2019[['arr_delay',var]]
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="arr_delay", data=data)
fig.axis(ymin=-100, ymax=100)

plt.savefig(f'../Images/Arrival_delays_monthly_boxplot_{session}.png')

In [ ]:
df_clean_2019.describe()

In [ ]:
pass_columns = sorted(list(df_pa_init.columns))
# pass_columns

In [ ]:
# df['origin_city_name'].value_counts()

There doesn't appear to be an observable trend.

Perhaps if we isolate the flights with a single country as a destination, for example the US:

In [ ]:
df_clean['origin_region_code'] = df_clean['origin_city_name'].str[-2:]
df_clean['dest_region_code'] = df_clean['dest_city_name'].str[-2:]


In [ ]:
us_states = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO'
  , 'CT', 'DE', 'DC', 'FL', 'GA', 'HI'
  , 'ID', 'IL', 'IN', 'IA', 'KS', 'KY'
  , 'LA', 'ME', 'MD', 'MA', 'MI', 'MN'
  , 'MS', 'MO', 'MT', 'NE', 'NV', 'NH'
  , 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH'
  , 'OK', 'OR', 'PA', 'RI', 'SC', 'SD'
  , 'TN', 'TX', 'UT', 'VT', 'VA', 'WA'
  , 'WV', 'WI', 'WY'
]


In [ ]:
df_us = df_clean[df_clean['dest_region_code'].isin(us_states)]
df_us.head()

In [ ]:
# month
var = 'month'
data = df_us[['arr_delay',var]]
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="arr_delay", data=data)
fig.axis(ymin=-60, ymax=60)

plt.savefig(f'../Images/Arr_delays_monthly_boxplot_US_{session}.png')

In [ ]:
df_us[['month', 'arr_delay']].groupby('month').median().sort_values('arr_delay')

#### **Task 3**: Does the weather affect the delay? 
Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:

[Local Historical Weather API, WWO](https://www.worldweatheronline.com/weather-api/api/docs/historical-weather-api.aspx)
- sunny
- cloudy
- rainy
- snow.
Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?

In [ ]:
# NOTE: would like to increase location precision later with lat/long coords of the dest_airport_id later
# For now will approximate with dest_city_name and 
dest_cities = list(df_clean['dest_city_name'])
arr_date = list(df_clean['date'])
arr_time = list(df_clean['arr_time'])

The `arr_time_code` number will be used to index the correct weather type according to the flight arr_time:


In [ ]:
arr_time_code = []
for i in arr_time:
    if i <= 150:
      arr_time_code.append(0)
    elif i <= 450:
      arr_time_code.append(1)
    elif i <= 750:
      arr_time_code.append(2)
    elif i <= 1050:
      arr_time_code.append(3)
    elif i <= 1350:
      arr_time_code.append(4)
    elif i <= 1650:
      arr_time_code.append(5)
    elif i <= 1950:
      arr_time_code.append(6)
    elif i <= 2250:
      arr_time_code.append(7)
    else:
      arr_time_code.append(0)

In [ ]:
print(dest_cities[0])
print(arr_date[0])
print(arr_time[0])
print(arr_time_code[0])

In [ ]:
def WWO_API_weather_type(city, date, time):
  '''
  input:
  output:  
  '''
  api_key = os.environ['WEATHER_API_KEY']
  params = {
    'q': city
    , 'date': date
    , 'format': 'json'
    , 'key': api_key
  }

  wwo_url = f'https://api.worldweatheronline.com/premium/v1/past-weather.ashx?'
  wwoHxWeather_json = requests.get(wwo_url, params=params).json()

  list_of_dict = []
  dest_site = wwoHxWeather_json['data']['weather'][0]['hourly']

  weather_dict = {
      'weather_type':   dest_site[time]['weatherDesc'][0]['value']
  }
  list_of_dict.append(weather_dict)
  return pd.DataFrame(list_of_dict)

    

In [ ]:
test_weather_desc = WWO_API_weather_type('Aberdeen, SD', '2018-01-01', 4)
test_weather_desc

In [ ]:
# Create list of tiny dfs:
weather_type_list = []
for i in range(df_clean.shape[0]):
  city_x = dest_cities[i]
  date_x = arr_date[i]
  time_x = arr_time_code[i]
  x = WWO_API_weather_type(city_x, date_x, time_x)
  weather_type_list.append(x)


I stopped the API function loop after 119min which only accumulated approx 21.8% of our data.

In [ ]:
# Confirm equal lengths:  
print(len(weather_type_list))
print(len(dest_cities))

In [ ]:
# Concatenate the list of dfs to one:

df_weather_type = pd.DataFrame()
df_x = pd.DataFrame()

for x in weather_type_list:
  df_x = pd.concat([df_weather_type, x])
  df_weather_type = df_x

In [ ]:
df_clean.shape

In [ ]:
# Create a truncated version of the df_clean to at least save what we have:
temp = df_clean.copy()
temp = temp.reset_index()
temp.shape

In [ ]:
df_clean_trunc = temp.loc[0:19702, :]
df_clean_trunc.shape

In [ ]:
df_weather_type.head(10)

In [ ]:
len(list(df_weather_type['weather_type']))

In [ ]:
# Add the new column to working df:
df_clean_trunc['weather'] = list(df_weather_type['weather_type'])

In [ ]:
# Confirm the weather type in df:
df_clean_trunc.weather.head()

##### SAVE new version of df_clean with weather_types:

In [ ]:
df_clean_trunc.to_csv(f'../data/flights_clean_df{session}.csv', index= False)

In [ ]:
# 
var = 'weather'
data = df_clean_trunc[['arr_delay',var]]

f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="arr_delay", data=data)
fig.axis(ymin=-60, ymax=60)

plt.savefig(f'../Images/Arr_delays_weather_type_boxplot_{session}.png')

In [ ]:
def WWO_API_aux_weather(city, date, time):
  '''
  input: This function requires 3 input variables. 
    1. city: acceptable format includes:         # string data type
        • City Name
        • City Name, State (US only)
        • City Name, State, Country
        • City Name, Country
        • IP address: XXX.XXX.XXX.XXX
        • Postal Code (UK or Canada), Zipcode (US)
        • Latitude and longitude in decimal degrees
    2. date: 
  output:  
  '''
  api_key = os.environ['WEATHER_API_KEY']
  params = {
    'q': city
    , 'date': date
    , 'format': 'json'
    , 'key': api_key
  }

  wwo_url = f'https://api.worldweatheronline.com/premium/v1/past-weather.ashx?'
  wwoHxWeather_json = requests.get(wwo_url, params=params).json()

  list_of_dict = []
  for i in wwoHxWeather_json['data']['weather'][0]:
    weather = wwoHxWeather_json['data']['weather'][0]
    hourly = wwoHxWeather_json['data']['weather'][0]['hourly']

    weather_dict = {
          'max_temp_C':           weather['maxtempC']
        , 'min_temp_C':           weather['mintempC']
        , 'avg_temp_C':           weather['avgtempC']
        , 'total_snow_cm':        weather['totalSnow_cm']
        , 'sun_hour':             weather['sunHour']
        , 'uv_index':             weather['uvIndex']
        , 'arr_wind_chill_C':     hourly[time]['WindChillC']
        , 'arr_wind_gust_Kmph':   hourly[time]['WindGustKmph']
        , 'arr_cloud_cover':      hourly[time]['cloudcover']
        , 'arr_precip_MM':        hourly[time]['precipMM']
        , 'arr_pressure':         hourly[time]['pressure']
        , 'arr_temp_C':           hourly[time]['tempC']
        , 'arr_time':             hourly[time]['time']
        , 'arr_uv_index':         hourly[time]['uvIndex']
        , 'arr_visibility':       hourly[time]['visibility']
        , 'arr_weather_code':     hourly[time]['weatherCode']
        , 'arr_wind_dir_16Point': hourly[time]['winddir16Point']
        , 'arr_wind_dir_degree':  hourly[time]['winddirDegree']
        , 'arr_wind_speed_Kmph':  hourly[time]['windspeedKmph']
        , 'arr_weather_type':     hourly[time]['weatherDesc'][0]['value']
    }
  list_of_dict.append(weather_dict)
  return pd.DataFrame(list_of_dict)

    

##### TEST API function:

In [ ]:
# NOTE: would like to increase location precision later with lat/long coords of the dest_airport_id later
# For now will approximate with dest_city_name and state code.
dest_cities = list(df_clean['dest_city_name'])
arr_date = list(df_clean['date'])
arr_time = list(df_clean['arr_time'])

In [ ]:
test_aux = WWO_API_aux_weather('Aberdeen, SD', '2018-01-01', 4)
test_aux

In [ ]:
df_test = pd.DataFrame(test['data']['weather'])
list(df_test.columns)

In [ ]:
hourly_columns = sorted(list(df_test_hourly.columns))

In [ ]:
df_test_hourly = pd.json_normalize(df_test['hourly'][0])
df_test_hourly

In [ ]:
arr_time_code = []
for i in arr_time:
    if i <= 150:
      arr_time_code.append(0)
    elif i <= 450:
      arr_time_code.append(1)
    elif i <= 750:
      arr_time_code.append(2)
    elif i <= 1050:
      arr_time_code.append(3)
    elif i <= 1350:
      arr_time_code.append(4)
    elif i <= 1650:
      arr_time_code.append(5)
    elif i <= 1950:
      arr_time_code.append(6)
    elif i <= 2250:
      arr_time_code.append(7)
    else:
      arr_time_code.append(0)

##### TEST API function:

In [ ]:
test = WorldWeatherOnlineAPI('Aberdeen, SD', '2018-01-01', 6)

In [ ]:
df_test = pd.DataFrame(test['data']['weather'])
list(df_test.columns)

In [ ]:
hourly_columns = sorted(list(df_test_hourly.columns))

In [ ]:
time_dict{
  0:[0, 150]
  1:[151, 450]
  2:[451, 750]
  3:[751, 1050]
  4:[1051, 1350]
  5:[1351, 1650]
  6:[1651, 1950]
  7:[1951, 2250]
  0:[2251, 2400]

}

In [ ]:
df_test_hourly = pd.json_normalize(df_test['hourly'][0])
df_test_hourly

In [ ]:
df_test_maxtemp['weatherDesc']

In [ ]:
test['data']
test['data']['weather']
test['data']['weather'][0]


In [ ]:
for i in range(df_clean.shape[0]):
  


#### **Task 4**: How taxi times changing during the day? Does higher traffic lead to bigger taxi times?

#### **Task 5**: What is the average percentage of delays that is already created before departure? (aka are arrival delays caused by departure delays?) Are airlines able to lower the delay during the flights?

In [107]:
flights_usa[['fl_date']] = flights_usa[['fl_date']].apply(pd.to_datetime)
flights_usa['fl_date']

0       2018-10-05
1       2018-10-05
2       2018-10-05
3       2018-10-05
4       2018-10-05
           ...    
99995   2019-10-09
99996   2019-10-09
99997   2019-10-09
99998   2019-10-09
99999   2019-10-09
Name: fl_date, Length: 100000, dtype: datetime64[ns]

In [108]:
flights_usa['year'] = flights_usa['fl_date'].dt.year
flights_usa['month'] = flights_usa['fl_date'].dt.month

In [109]:
#See the distributions
flights_usa[['year', 'month']].value_counts().sort_index(ascending=False)

year  month
2019  12       6918
      10       2028
      9        6557
      7        7118
      5        6981
      4        6759
      3        6882
      1        6215
2018  10       5590
      9        6283
      8        6941
      7        6951
      5        6633
      4        6577
      2        5669
      1        5898
dtype: int64

In [110]:
flights_usa['state'] = flights_usa['origin_city_name'].str[-2:]
flights_usa['late_arr'] = (flights_usa['arr_delay'] > 0).astype(int)
flights_usa['late_dep'] = (flights_usa['dep_delay'] > 0).astype(int)
flights_usa

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,year,month,state,late_arr,late_dep
0,2018-10-05,WN,WN,WN,179,WN,N8312C,179,12953,LGA,...,NaN,NaN,NaN,NaN,NaN,2018,10,NY,0,0
1,2018-10-05,WN,WN,WN,1106,WN,N219WN,1106,12953,LGA,...,28.0,NaN,NaN,NaN,NaN,2018,10,NY,1,1
2,2018-10-05,WN,WN,WN,1579,WN,N8535S,1579,13232,MDW,...,NaN,NaN,NaN,NaN,NaN,2018,10,IL,1,1
3,2018-10-05,WN,WN,WN,2136,WN,N436WN,2136,13342,MKE,...,294.0,NaN,NaN,NaN,NaN,2018,10,WI,1,1
4,2018-10-05,WN,WN,WN,2423,WN,N221WN,2423,14057,PDX,...,NaN,NaN,NaN,NaN,NaN,2018,10,OR,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2019-10-09,F9,F9,F9,2163,F9,N352FR,2163,12889,LAS,...,NaN,NaN,NaN,NaN,NaN,2019,10,NV,0,1
99996,2019-10-09,AA,AA_CODESHARE,AA,5772,YV,N911FJ,5772,11298,DFW,...,NaN,NaN,NaN,NaN,NaN,2019,10,TX,0,0
99997,2019-10-09,AA,AA_CODESHARE,AA,5815,YV,N946LR,5815,11298,DFW,...,0.0,NaN,NaN,NaN,NaN,2019,10,TX,1,1
99998,2019-10-09,AA,AA_CODESHARE,AA,5492,OH,N593NN,5492,11042,CLE,...,NaN,NaN,NaN,NaN,NaN,2019,10,OH,0,0


In [111]:
flights_usa['speed'] = flights_usa['distance']/flights_usa['air_time']
no_dep_delay = flights_usa[flights_usa['late_dep'] == 0]
yes_dep_delay = flights_usa[flights_usa['late_dep'] == 1]

In [112]:
#If there is no departure delay, there is a 15% chance of late arrival
no_dep_delay['late_arr'].mean()

0.14478611819823045

In [113]:
#If there is a departure delay, there is a 73% chance of late arrival
yes_dep_delay['late_arr'].mean()

0.7272038087151651

#### **Task 6**: How many states cover 50% of US air traffic? 

In [114]:
top_8 = flights_usa['state'].value_counts().head(8)
top_8

CA    10876
TX    10087
FL     7299
IL     6334
GA     5352
NY     5231
NC     4531
CO     3864
Name: state, dtype: int64

In [115]:
total_flight = flights_usa['origin_city_name'].count()
total_flight

100000

In [116]:
#These 8 states cover 53% of the flight
top_8.sum()/total_flight

0.53574

#### **Task 7**: Test the hypothesis whether planes fly faster when there is the departure delay? 

In [117]:
#Mean of planes speed without departure delay
no_dep_delay['speed'].mean()

6.591761527423723

In [118]:
#Mean of planes speed with departure delay
yes_dep_delay['speed'].mean()

6.749625050583639

#### **Task 8**: When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?

Short-haul is a flight lasting anywhere from 30 minutes to 3 hours. Medium-haul is defined by flights lasting between 3-6 hours. And lastly, long-haul flights are those that extend beyond 6 hours.

In [119]:
flights_usa['dep_time_hour'] = flights_usa['dep_time']//100

In [120]:
flights_clean_airtime = flights_usa[flights_usa['air_time'].notna()]
flights_clean_airtime

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,total_add_gtime,longest_add_gtime,no_name,year,month,state,late_arr,late_dep,speed,dep_time_hour
0,2018-10-05,WN,WN,WN,179,WN,N8312C,179,12953,LGA,...,NaN,NaN,NaN,2018,10,NY,0,0,6.945455,16.0
1,2018-10-05,WN,WN,WN,1106,WN,N219WN,1106,12953,LGA,...,NaN,NaN,NaN,2018,10,NY,1,1,6.775701,12.0
2,2018-10-05,WN,WN,WN,1579,WN,N8535S,1579,13232,MDW,...,NaN,NaN,NaN,2018,10,IL,1,1,7.231405,15.0
3,2018-10-05,WN,WN,WN,2136,WN,N436WN,2136,13342,MKE,...,NaN,NaN,NaN,2018,10,WI,1,1,7.286307,17.0
4,2018-10-05,WN,WN,WN,2423,WN,N221WN,2423,14057,PDX,...,NaN,NaN,NaN,2018,10,OR,0,0,7.438356,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2019-10-09,F9,F9,F9,2163,F9,N352FR,2163,12889,LAS,...,NaN,NaN,NaN,2019,10,NV,0,1,5.363636,18.0
99996,2019-10-09,AA,AA_CODESHARE,AA,5772,YV,N911FJ,5772,11298,DFW,...,NaN,NaN,NaN,2019,10,TX,0,0,5.886792,12.0
99997,2019-10-09,AA,AA_CODESHARE,AA,5815,YV,N946LR,5815,11298,DFW,...,NaN,NaN,NaN,2019,10,TX,1,1,7.537500,21.0
99998,2019-10-09,AA,AA_CODESHARE,AA,5492,OH,N593NN,5492,11042,CLE,...,NaN,NaN,NaN,2019,10,OH,0,0,6.056338,17.0


In [121]:
flights_small = flights_clean_airtime[flights_clean_airtime['air_time']<180]
flights_small

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,total_add_gtime,longest_add_gtime,no_name,year,month,state,late_arr,late_dep,speed,dep_time_hour
0,2018-10-05,WN,WN,WN,179,WN,N8312C,179,12953,LGA,...,NaN,NaN,NaN,2018,10,NY,0,0,6.945455,16.0
1,2018-10-05,WN,WN,WN,1106,WN,N219WN,1106,12953,LGA,...,NaN,NaN,NaN,2018,10,NY,1,1,6.775701,12.0
4,2018-10-05,WN,WN,WN,2423,WN,N221WN,2423,14057,PDX,...,NaN,NaN,NaN,2018,10,OR,0,0,7.438356,11.0
5,2018-10-05,WN,WN,WN,1744,WN,N452WN,1744,14107,PHX,...,NaN,NaN,NaN,2018,10,AZ,1,1,7.846154,18.0
6,2018-10-05,WN,WN,WN,951,WN,N8668A,951,14107,PHX,...,NaN,NaN,NaN,2018,10,AZ,1,1,7.154762,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2019-10-09,F9,F9,F9,2163,F9,N352FR,2163,12889,LAS,...,NaN,NaN,NaN,2019,10,NV,0,1,5.363636,18.0
99996,2019-10-09,AA,AA_CODESHARE,AA,5772,YV,N911FJ,5772,11298,DFW,...,NaN,NaN,NaN,2019,10,TX,0,0,5.886792,12.0
99997,2019-10-09,AA,AA_CODESHARE,AA,5815,YV,N946LR,5815,11298,DFW,...,NaN,NaN,NaN,2019,10,TX,1,1,7.537500,21.0
99998,2019-10-09,AA,AA_CODESHARE,AA,5492,OH,N593NN,5492,11042,CLE,...,NaN,NaN,NaN,2019,10,OH,0,0,6.056338,17.0


In [122]:
flights_small['dep_time_hour'].value_counts().head(10)

6.0     5393
11.0    5326
17.0    5324
12.0    5280
8.0     5232
10.0    5161
15.0    5132
7.0     5046
14.0    4955
9.0     4904
Name: dep_time_hour, dtype: int64

In [123]:
flights_medium = flights_clean_airtime[flights_clean_airtime['air_time']>=180]
flights_medium = flights_medium[flights_medium['air_time']<360]
flights_medium

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,total_add_gtime,longest_add_gtime,no_name,year,month,state,late_arr,late_dep,speed,dep_time_hour
2,2018-10-05,WN,WN,WN,1579,WN,N8535S,1579,13232,MDW,...,NaN,NaN,NaN,2018,10,IL,1,1,7.231405,15.0
3,2018-10-05,WN,WN,WN,2136,WN,N436WN,2136,13342,MKE,...,NaN,NaN,NaN,2018,10,WI,1,1,7.286307,17.0
35,2018-10-05,NK,NK,NK,468,NK,N526NK,468,12889,LAS,...,NaN,NaN,NaN,2018,10,NV,0,0,7.978723,18.0
78,2018-10-05,DL,DL,DL,33,DL,N338DN,33,10397,ATL,...,NaN,NaN,NaN,2018,10,GA,0,0,7.850622,21.0
80,2018-10-05,DL,DL,DL,430,DL,N717TW,430,12478,JFK,...,NaN,NaN,NaN,2018,10,NY,0,0,7.473988,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99946,2019-10-09,HA,HA,HA,48,HA,N212HA,48,12173,HNL,...,NaN,NaN,NaN,2019,10,HI,0,1,8.364583,12.0
99970,2019-10-09,DL,DL,DL,1375,DL,N376DN,1375,14679,SAN,...,NaN,NaN,NaN,2019,10,CA,0,0,8.841121,7.0
99977,2019-10-09,DL,DL,DL,1858,DL,N920DU,1858,10397,ATL,...,NaN,NaN,NaN,2019,10,GA,1,1,7.479021,20.0
99983,2019-10-09,DL,DL,DL,2193,DL,N917DU,2193,13487,MSP,...,NaN,NaN,NaN,2019,10,MN,0,0,7.441489,6.0


In [124]:
flights_medium['dep_time_hour'].value_counts().head(10)

7.0     965
8.0     941
6.0     841
9.0     787
17.0    735
11.0    729
12.0    725
15.0    687
16.0    684
13.0    666
Name: dep_time_hour, dtype: int64

In [125]:
flights_long = flights_clean_airtime[flights_clean_airtime['air_time']>=360]
flights_long

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,total_add_gtime,longest_add_gtime,no_name,year,month,state,late_arr,late_dep,speed,dep_time_hour
239,2018-10-06,UA,UA,UA,477,UA,N41135,477,10721,BOS,...,NaN,NaN,NaN,2018,10,MA,1,0,6.811083,11.0
285,2018-10-06,AS,AS,AS,139,AS,N535AS,139,13930,ORD,...,NaN,NaN,NaN,2018,10,IL,1,0,7.354005,9.0
417,2018-10-07,HA,HA,HA,35,HA,N580HA,35,14107,PHX,...,NaN,NaN,NaN,2018,10,AZ,0,0,7.948229,7.0
730,2018-10-09,UA,UA,UA,200,UA,N768UA,200,12016,GUM,...,NaN,NaN,NaN,2018,10,TT,0,0,9.007109,6.0
1117,2018-10-10,AA,AA,AA,241,AA,N133AN,241,10721,BOS,...,NaN,NaN,NaN,2018,10,MA,1,0,7.037736,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98704,2019-10-04,UA,UA,UA,1820,UA,N14115,1820,10721,BOS,...,NaN,NaN,NaN,2019,10,MA,1,0,7.212707,8.0
99289,2019-10-06,B6,B6,B6,415,B6,N976JT,415,12478,JFK,...,NaN,NaN,NaN,2019,10,NY,0,0,7.163435,9.0
99700,2019-10-08,UA,UA,UA,2307,UA,N12116,2307,11292,DEN,...,NaN,NaN,NaN,2019,10,CO,0,0,8.167464,9.0
99945,2019-10-09,HA,HA,HA,7,HA,N379HA,7,12889,LAS,...,NaN,NaN,NaN,2019,10,NV,1,1,7.650970,9.0


In [126]:
flights_long['dep_time_hour'].value_counts().head(10)

11.0    47
9.0     45
10.0    39
7.0     37
17.0    35
19.0    31
16.0    31
15.0    29
18.0    24
8.0     19
Name: dep_time_hour, dtype: int64

#### **Task 9**: Find the top 10 the bussiest airports. Does the biggest number of flights mean that the biggest number of passengers went through the particular airport? How much traffic do these 10 airports cover?

In [149]:
flights_usa['origin_airport_id'].value_counts().head(15)

10397    5038
13930    4974
11298    3820
11292    3476
12892    3140
11057    3089
14747    2399
14107    2279
11433    2266
12266    2246
14771    2175
12953    2141
12889    2040
13487    1993
14100    1952
Name: origin_airport_id, dtype: int64

In [134]:
flights_usa.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name', 'year', 'month', 'state', 'late_arr',
       'late_dep', 'speed', 'dep_time_hour'],
      dtype='object')

In [140]:
df_pa_init    = passengers_initial_300000_records.copy()

In [162]:
df_pa_init

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,...,dest_city_name,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,month,year,distance_group,class
0,3,3,3600,18,0,210,41,81,166,160,...,"Olga Bay, AK",US,United States,0,40,4,10,2019,1,F
1,3,3,3000,15,3,24,0,39,67,61,...,"Old Harbor, AK",US,United States,0,79,3,10,2019,1,F
2,3,3,3600,18,1,0,0,93,167,161,...,"Kodiak Island, AK",US,United States,0,40,4,10,2019,1,F
3,3,3,3600,18,4,175,86,14,38,32,...,"Seal Bay, AK",US,United States,0,40,4,10,2019,1,F
4,3,3,3600,18,0,209,41,10,32,26,...,"Lazy Bay, AK",US,United States,0,40,4,10,2019,1,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,0,22,29700,132,60,929,6161,67,929,776,...,"Kongiganak, AK",US,United States,0,35,3,6,2018,1,F
299996,0,21,71400,189,119,1325,8953,67,635,537,...,"Kongiganak, AK",US,United States,4,416,3,6,2018,1,F
299997,0,1,3400,9,0,0,0,67,32,28,...,"Kongiganak, AK",US,United States,4,416,3,6,2018,1,L
299998,0,32,43200,192,94,615,2556,15,468,300,...,"Akiachak, AK",US,United States,0,35,3,6,2018,1,F


In [169]:
top_10_busiest = df_pa_init[['passengers', 'origin_airport_id']].groupby(['origin_airport_id']).sum().sort_values(by='passengers', ascending = False).head(10)
top_10_busiest

,passengers
origin_airport_id,
10397,52600069
13930,37614331
12892,35368279
11298,34922835
11292,32818645
11057,23912817
14771,23665636
12889,23206372
14747,23186288


#### **Task 10**: Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?